In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from sklearn.metrics import jaccard_score

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from scipy.spatial.distance import jaccard

import warnings
warnings.filterwarnings('ignore')

# Loading the data

In [2]:
#Using the latest data

df = pd.read_csv('Cleaned_data_GK_V3.csv')
df

,ID,Title,Pillar,Instructor,Categories,Difficulty,Difficulty_Num,Duration,Duration_Num,Require_Movement,...,Type,Audio,Recipe,Text,Video,Visualization,Activity,Beauty,Nutrition,Rest
0,14,a comprehensive guide to essential vitamins an...,nutrition,iain mcbride,education,intermediate,2,6-15 min,3,no,...,"text, video",0,0,1,1,0,0,0,1,0
1,122,a comprehensive guide to nutritional balance,nutrition,jess vergeer,education,intermediate,2,6-15 min,3,no,...,"text, video",0,0,1,1,0,0,0,1,0
2,352,a comprehensive guide to nutritional basics,nutrition,jess vergeer,education,intermediate,2,6-15 min,3,no,...,"text, video",0,0,1,1,0,0,0,1,0
3,280,a comprehensive guide to nutritional essentials,nutrition,jess vergeer,education,intermediate,2,6-15 min,3,no,...,"text, video",0,0,1,1,0,0,0,1,0
4,174,a comprehensive guide to vitamins and minerals,nutrition,iain mcbride,education,intermediate,2,6-15 min,3,no,...,"text, video",0,0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,231,yoga for the lower back,rest,julie chinmayi,mind and body,beginner,1,6-15 min,3,no,...,video,0,0,0,1,0,0,0,0,1
424,99,yoga foundations - concentration,rest,julie chinmayi,mind and body,beginner,1,6-15 min,3,no,...,video,0,0,0,1,0,0,0,0,1
425,323,yoga foundations - finding balance,rest,julie chinmayi,mind and body,intermediate,2,15 - 30 min,4,no,...,video,0,0,0,1,0,0,0,0,1
426,383,yoga foundations - non-attachment,rest,julie chinmayi,mind and body,beginner,1,6-15 min,3,no,...,video,0,0,0,1,0,0,0,0,1


In [4]:
len(df['Title'].unique())

428

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    428 non-null    int64  
 1   Title                 428 non-null    object 
 2   Pillar                428 non-null    object 
 3   Instructor            428 non-null    object 
 4   Categories            428 non-null    object 
 5   Difficulty            428 non-null    object 
 6   Difficulty_Num        428 non-null    int64  
 7   Duration              428 non-null    object 
 8   Duration_Num          428 non-null    int64  
 9   Require_Movement      428 non-null    object 
 10  Require_Movement_Num  428 non-null    float64
 11  Tags                  428 non-null    object 
 12  Source                428 non-null    object 
 13  Type                  428 non-null    object 
 14  Audio                 428 non-null    int64  
 15  Recipe                4

In [37]:
df.isnull().sum()

ID                      0
Title                   0
Pillar                  0
Instructor              0
Categories              0
Difficulty              0
Difficulty_Num          0
Duration                0
Duration_Num            0
Require_Movement        0
Require_Movement_Num    0
Tags                    0
Source                  0
Type                    0
Audio                   0
Recipe                  0
Text                    0
Video                   0
Visualization           0
Activity                0
Beauty                  0
Nutrition               0
Rest                    0
dtype: int64

In [9]:
df['Title'].unique()

array(['a comprehensive guide to essential vitamins and minerals',
       'a comprehensive guide to nutritional balance',
       'a comprehensive guide to nutritional basics',
       'a comprehensive guide to nutritional essentials',
       'a comprehensive guide to vitamins and minerals',
       'advanced hiit workout challenge',
       'advanced strength training routine', 'advanced yoga flow',
       'affirmations for boosting self-confidence',
       'affirmations for building confidence',
       'affirmations for cultivating self-love',
       'affirmations for embracing self-love',
       'affirmations for self-esteem',
       'affirmations for a positive morning',
       'an in-depth guide to essential nutrients',
       'an in-depth guide to vitamins and minerals',
       'begin your day with empowering affirmations',
       'body positivity affirmations',
       'boosting positivity with daily affirmations',
       'building self-esteem with positive affirmations',
       'car

In [3]:
df['Pillar'].unique()

array(['nutrition', 'activity', 'activity, rest', 'beauty',
       'beauty, rest', 'rest', 'beauty, nutrition', 'beauty, activity',
       'rest, activity'], dtype=object)

In [5]:
df['Instructor'].unique()

array(['iain mcbride', 'jess vergeer', 'mark boateng', 'bao',
       'jess vergeer, iain mcbride', 'julie chinmayi', 'sophon',
       'walerija', 'ola kwiatkowska', 'nitti'], dtype=object)

In [8]:
df['Categories'].unique()

array(['education', 'fitness', 'affirmation',
       'meditation and mindfulness', 'meditation, mind and body',
       'nature sounds and ambient music', 'recipes',
       'get to know the technique',
       'get started, short and sweet, fitness mojo',
       'get started, just your body, fitness mojo',
       'get started, short and sweet', 'just your body, fitness mojo',
       'on the floor, get started', 'short and sweet, healthy desserts',
       'push yourself, just your body, get energised', 'check why',
       'goe academy', 'take a breath, mind and body',
       'take abreath, mind and body, relax', 'take a breath, relax',
       'take a breath, relax, mind and body',
       'short and sweet, healthy desserts, quick and easy',
       'stretching, yoga',
       'just your body, on the floor, sweet and short',
       'just your body, push yourself', 'take a breath',
       'mind and body, my evening routine',
       'push yourself, just your body', 'get energised',
       'shor

series & categories have similar values

In [7]:
df['Tags'].unique()

array(['Guide, Wellness, Education, Comprehensive, Nutrition, Minerals, Essential, Vitamins, Health',
       'Guide, Wellness, Education, Comprehensive, Nutrition, Balance, Health',
       'Guide, Wellness, Education, Comprehensive, Nutrition, Basics, Health',
       'Guide, Wellness, Education, Essential, Comprehensive, Nutrition, Health',
       'Guide, Wellness, Education, Comprehensive, Nutrition, Minerals, Vitamins, Health',
       'Challenge, Hiit, Fitness', 'Training, Strength, Routine',
       'Strength, Flexibility, Balance',
       'Growth, Self-confidence, Affirmation, Positive, Personal development, Boosting, Empowerment, Esteem',
       'Growth, Self-esteem, Positive, Building, Personal development, Confidence, Motivation, Empowerment',
       'Growth, Affirmation, Self-love, Positive, Personal development, Cultivating, Empowerment, Esteem',
       'Growth, Affirmation, Empowerment, Self-love, Positive, Personal development, Confidence, Embracing',
       'Positive, Confid

# Preprocessing

In [3]:
#I have combined Pillar, Instructor, Categories, Difficulty, Duration, Require movement, Tags & Type

df['feature'] = df['Pillar'] + ' '+ df['Instructor']+ ' ' + df['Categories'] + ' '+ df['Difficulty'] + ' '+ df['Duration'] + df['Require_Movement'] + ' ' + df['Tags'] + ' ' + df['Type']
df

,ID,Title,Pillar,Instructor,Categories,Difficulty,Difficulty_Num,Duration,Duration_Num,Require_Movement,...,Audio,Recipe,Text,Video,Visualization,Activity,Beauty,Nutrition,Rest,feature
0,14,a comprehensive guide to essential vitamins an...,nutrition,iain mcbride,education,intermediate,2,6-15 min,3,no,...,0,0,1,1,0,0,0,1,0,nutrition iain mcbride education intermediate ...
1,122,a comprehensive guide to nutritional balance,nutrition,jess vergeer,education,intermediate,2,6-15 min,3,no,...,0,0,1,1,0,0,0,1,0,nutrition jess vergeer education intermediate ...
2,352,a comprehensive guide to nutritional basics,nutrition,jess vergeer,education,intermediate,2,6-15 min,3,no,...,0,0,1,1,0,0,0,1,0,nutrition jess vergeer education intermediate ...
3,280,a comprehensive guide to nutritional essentials,nutrition,jess vergeer,education,intermediate,2,6-15 min,3,no,...,0,0,1,1,0,0,0,1,0,nutrition jess vergeer education intermediate ...
4,174,a comprehensive guide to vitamins and minerals,nutrition,iain mcbride,education,intermediate,2,6-15 min,3,no,...,0,0,1,1,0,0,0,1,0,nutrition iain mcbride education intermediate ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,231,yoga for the lower back,rest,julie chinmayi,mind and body,beginner,1,6-15 min,3,no,...,0,0,0,1,0,0,0,0,1,rest julie chinmayi mind and body beginner 6-1...
424,99,yoga foundations - concentration,rest,julie chinmayi,mind and body,beginner,1,6-15 min,3,no,...,0,0,0,1,0,0,0,0,1,rest julie chinmayi mind and body beginner 6-1...
425,323,yoga foundations - finding balance,rest,julie chinmayi,mind and body,intermediate,2,15 - 30 min,4,no,...,0,0,0,1,0,0,0,0,1,rest julie chinmayi mind and body intermediate...
426,383,yoga foundations - non-attachment,rest,julie chinmayi,mind and body,beginner,1,6-15 min,3,no,...,0,0,0,1,0,0,0,0,1,rest julie chinmayi mind and body beginner 6-1...


In [4]:
df2 = df[['Title', 'feature', 'Pillar', 'Instructor', 'Difficulty', 'Duration', 'Tags']]
df2.dropna(inplace = True)
df2.reset_index(drop = True, inplace = True)
df2

,Title,feature,Pillar,Instructor,Difficulty,Duration,Tags
0,a comprehensive guide to essential vitamins an...,nutrition iain mcbride education intermediate ...,nutrition,iain mcbride,intermediate,6-15 min,"Guide, Wellness, Education, Comprehensive, Nut..."
1,a comprehensive guide to nutritional balance,nutrition jess vergeer education intermediate ...,nutrition,jess vergeer,intermediate,6-15 min,"Guide, Wellness, Education, Comprehensive, Nut..."
2,a comprehensive guide to nutritional basics,nutrition jess vergeer education intermediate ...,nutrition,jess vergeer,intermediate,6-15 min,"Guide, Wellness, Education, Comprehensive, Nut..."
3,a comprehensive guide to nutritional essentials,nutrition jess vergeer education intermediate ...,nutrition,jess vergeer,intermediate,6-15 min,"Guide, Wellness, Education, Essential, Compreh..."
4,a comprehensive guide to vitamins and minerals,nutrition iain mcbride education intermediate ...,nutrition,iain mcbride,intermediate,6-15 min,"Guide, Wellness, Education, Comprehensive, Nut..."
...,...,...,...,...,...,...,...
423,yoga for the lower back,rest julie chinmayi mind and body beginner 6-1...,rest,julie chinmayi,beginner,6-15 min,Yoga
424,yoga foundations - concentration,rest julie chinmayi mind and body beginner 6-1...,rest,julie chinmayi,beginner,6-15 min,"Yoga, Yoga"
425,yoga foundations - finding balance,rest julie chinmayi mind and body intermediate...,rest,julie chinmayi,intermediate,15 - 30 min,"Yoga, Yoga, Deep"
426,yoga foundations - non-attachment,rest julie chinmayi mind and body beginner 6-1...,rest,julie chinmayi,beginner,6-15 min,"Emotional, Yoga"


In [5]:
#removing stopwords, punctuations & cleaning the features
#As there are no sentences or descriptions so I havent used lemmatization

stop = set(stopwords.words('english'))
punct = list(string.punctuation)
stop.update(punct)

def clean_features(feature):
    new = []
    fea = word_tokenize(feature)
    
    for word in fea:
        if word.lower() not in stop:
            new.append(word)
    return ' '.join(new)

In [6]:
df2['feature'] = df2['feature'].apply(clean_features)
df2.head()

,Title,feature,Pillar,Instructor,Difficulty,Duration,Tags
0,a comprehensive guide to essential vitamins an...,nutrition iain mcbride education intermediate ...,nutrition,iain mcbride,intermediate,6-15 min,"Guide, Wellness, Education, Comprehensive, Nut..."
1,a comprehensive guide to nutritional balance,nutrition jess vergeer education intermediate ...,nutrition,jess vergeer,intermediate,6-15 min,"Guide, Wellness, Education, Comprehensive, Nut..."
2,a comprehensive guide to nutritional basics,nutrition jess vergeer education intermediate ...,nutrition,jess vergeer,intermediate,6-15 min,"Guide, Wellness, Education, Comprehensive, Nut..."
3,a comprehensive guide to nutritional essentials,nutrition jess vergeer education intermediate ...,nutrition,jess vergeer,intermediate,6-15 min,"Guide, Wellness, Education, Essential, Compreh..."
4,a comprehensive guide to vitamins and minerals,nutrition iain mcbride education intermediate ...,nutrition,iain mcbride,intermediate,6-15 min,"Guide, Wellness, Education, Comprehensive, Nut..."


In [8]:
df2.to_csv('title_features.csv', index = False)

In [7]:
#Calculating td-idf vectorizer
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,3))
tf_matrix = tf.fit_transform(df2['feature'])
tf_matrix.toarray().shape

(428, 2463)

# Calculating recommendations based on metrics

In [9]:
#cosine function ranges between -1 to 1
#the closer to 1, more similar the two vectors

def get_cos_sim_matrix(tf_matrix):
    return cosine_similarity(tf_matrix, tf_matrix)

In [10]:
#euclidean distance ranges between 0 to infinity
#lesser the distance, more similar the two vectors

def get_euc_dist_matrix(tf_matrix):
    return euclidean_distances(tf_matrix, tf_matrix)

In [11]:
#pearson coefficient ranges between -1 to 1
#the closer to 1, more similar the two vectors

def get_pearson_coef_matrix(tf_matrix):
    tf_matrix_array  = tf_matrix.toarray()
    return np.corrcoef(tf_matrix_array)

In [12]:
def get_manh_dist_matrix(tf_matrix):
    return manhattan_distances(tf_matrix, tf_matrix)

In [13]:
#defining a function that can recommend similar number of titles based on the metric

def recommend_similar(title, num_to_be_recommended, metric):
    index = df2[df2['Title'] == title].index[0]
    
    if metric == 'cosine':
        sim_matrix = get_cos_sim_matrix(tf_matrix)
    elif metric == 'euclidean':
        sim_matrix = get_euc_dist_matrix(tf_matrix)
    elif metric == 'pearson' :
        sim_matrix = get_pearson_coef_matrix(tf_matrix)
    elif metric == 'manhattan':
        sim_matrix = get_manh_dist_matrix(tf_matrix)
    else:
        return "Invalid metric"
        
        
    sims = list(enumerate(sim_matrix[index]))
    
    if metric == 'euclidean' or metric == 'manhattan':
        sims = sorted(sims, key = lambda x:x[1], reverse = False)[0: num_to_be_recommended]
    else:
        sims = sorted(sims, key = lambda x:x[1], reverse = True)[0: num_to_be_recommended]
        
    rec_titles = []
    scores = []
    pillars = []
    instructors = []
    difficulty = []
    duration = []
    
    
    for i in range(0, len(sims)):
        ind = sims[i][0]
        rec_title = df2['Title'][ind]
        rec_titles.append(rec_title)
        score = sims[i][1]
        scores.append(score)
        pillars.append(df2['Pillar'][ind])
        instructors.append(df2['Instructor'][ind])
        difficulty.append(df2['Difficulty'][ind])
        duration.append(df2['Duration'][ind])
     
        
    res = pd.DataFrame({'Recommended_titles': rec_titles, f'{metric}': scores, 'Pillar': pillars, 'Instructor': instructors,
                       'Difficulty': difficulty, 'Duration': duration})
    return res   
    

# Example 1

In [14]:
title = 'a comprehensive guide to nutritional basics'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,a comprehensive guide to nutritional basics,1.000000,nutrition,jess vergeer,intermediate,6-15 min
1,a comprehensive guide to nutritional balance,0.737844,nutrition,jess vergeer,intermediate,6-15 min
2,a comprehensive guide to nutritional essentials,0.614514,nutrition,jess vergeer,intermediate,6-15 min
3,a comprehensive guide to vitamins and minerals,0.586634,nutrition,iain mcbride,intermediate,6-15 min
4,vitamins and minerals: a comprehensive guide,0.586634,nutrition,iain mcbride,intermediate,6-15 min
5,a comprehensive guide to essential vitamins an...,0.546920,nutrition,iain mcbride,intermediate,6-15 min
6,an in-depth guide to essential nutrients,0.519836,nutrition,jess vergeer,intermediate,6-15 min
7,understanding macronutrients,0.447353,nutrition,jess vergeer,intermediate,6-15 min
8,an in-depth guide to vitamins and minerals,0.427741,nutrition,"jess vergeer, iain mcbride",intermediate,6-15 min
9,understanding vitamins and minerals,0.355579,nutrition,jess vergeer,intermediate,6-15 min


In [54]:
title = 'a comprehensive guide to nutritional basics'
num_to_be_recommended = 10
metric = 'euclidean'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,euclidean,Pillar,Instructor,Difficulty,Duration
0,a comprehensive guide to nutritional basics,0.000000,nutrition,jess vergeer,intermediate,6-15 min
1,a comprehensive guide to nutritional balance,0.724094,nutrition,jess vergeer,intermediate,6-15 min
2,a comprehensive guide to nutritional essentials,0.878050,nutrition,jess vergeer,intermediate,6-15 min
3,a comprehensive guide to vitamins and minerals,0.909248,nutrition,iain mcbride,intermediate,6-15 min
4,vitamins and minerals: a comprehensive guide,0.909248,nutrition,iain mcbride,intermediate,6-15 min
5,a comprehensive guide to essential vitamins an...,0.951924,nutrition,iain mcbride,intermediate,6-15 min
6,an in-depth guide to essential nutrients,0.979963,nutrition,jess vergeer,intermediate,6-15 min
7,understanding macronutrients,1.051330,nutrition,jess vergeer,intermediate,6-15 min
8,an in-depth guide to vitamins and minerals,1.069822,nutrition,"jess vergeer, iain mcbride",intermediate,6-15 min
9,understanding vitamins and minerals,1.135272,nutrition,jess vergeer,intermediate,6-15 min


In [55]:
title = 'a comprehensive guide to nutritional basics'
num_to_be_recommended = 10
metric = 'pearson'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,pearson,Pillar,Instructor,Difficulty,Duration
0,a comprehensive guide to nutritional basics,1.000000,nutrition,jess vergeer,intermediate,6-15 min
1,a comprehensive guide to nutritional balance,0.733648,nutrition,jess vergeer,intermediate,6-15 min
2,a comprehensive guide to nutritional essentials,0.608385,nutrition,jess vergeer,intermediate,6-15 min
3,a comprehensive guide to vitamins and minerals,0.579750,nutrition,iain mcbride,intermediate,6-15 min
4,vitamins and minerals: a comprehensive guide,0.579750,nutrition,iain mcbride,intermediate,6-15 min
5,a comprehensive guide to essential vitamins an...,0.539165,nutrition,iain mcbride,intermediate,6-15 min
6,an in-depth guide to essential nutrients,0.511931,nutrition,jess vergeer,intermediate,6-15 min
7,understanding macronutrients,0.439923,nutrition,jess vergeer,intermediate,6-15 min
8,an in-depth guide to vitamins and minerals,0.417451,nutrition,"jess vergeer, iain mcbride",intermediate,6-15 min
9,understanding vitamins and minerals,0.346874,nutrition,jess vergeer,intermediate,6-15 min


In [59]:
title = 'a comprehensive guide to nutritional basics'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,a comprehensive guide to nutritional basics,0.000000,nutrition,jess vergeer,intermediate,6-15 min
1,a comprehensive guide to nutritional balance,2.593936,nutrition,jess vergeer,intermediate,6-15 min
2,a comprehensive guide to nutritional essentials,3.803194,nutrition,jess vergeer,intermediate,6-15 min
3,a comprehensive guide to vitamins and minerals,4.959529,nutrition,iain mcbride,intermediate,6-15 min
4,vitamins and minerals: a comprehensive guide,4.959529,nutrition,iain mcbride,intermediate,6-15 min
5,an in-depth guide to essential nutrients,5.208642,nutrition,jess vergeer,intermediate,6-15 min
6,a comprehensive guide to essential vitamins an...,5.673718,nutrition,iain mcbride,intermediate,6-15 min
7,understanding macronutrients,5.911641,nutrition,jess vergeer,intermediate,6-15 min
8,understanding vitamins and minerals,6.879981,nutrition,jess vergeer,intermediate,6-15 min
9,an in-depth guide to vitamins and minerals,7.044602,nutrition,"jess vergeer, iain mcbride",intermediate,6-15 min


# Example 2

In [60]:
title = 'yoga foundations - concentration'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,yoga foundations - concentration,1.000000,rest,julie chinmayi,beginner,6-15 min
1,yoga for cyclists by julie,0.801835,rest,julie chinmayi,beginner,6-15 min
2,yoga for grumpy knees,0.801835,rest,julie chinmayi,beginner,6-15 min
3,yoga for the lower back,0.801835,rest,julie chinmayi,beginner,6-15 min
4,yoga foundations - finding balance,0.553399,rest,julie chinmayi,intermediate,15 - 30 min
5,yoga foundations - non-attachment,0.540415,rest,julie chinmayi,beginner,6-15 min
6,yoga for concentration,0.509856,rest,julie chinmayi,intermediate,15 - 30 min
7,yoga for energy,0.509856,rest,julie chinmayi,intermediate,15 - 30 min
8,family friendly yoga with kids,0.474660,rest,julie chinmayi,beginner,6-15 min
9,yoga - the moon salutation,0.474660,rest,julie chinmayi,beginner,6-15 min


In [61]:
title = 'yoga foundations - concentration'
num_to_be_recommended = 10
metric = 'euclidean'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,euclidean,Pillar,Instructor,Difficulty,Duration
0,yoga foundations - concentration,0.000000,rest,julie chinmayi,beginner,6-15 min
1,yoga for cyclists by julie,0.629548,rest,julie chinmayi,beginner,6-15 min
2,yoga for grumpy knees,0.629548,rest,julie chinmayi,beginner,6-15 min
3,yoga for the lower back,0.629548,rest,julie chinmayi,beginner,6-15 min
4,yoga foundations - finding balance,0.945094,rest,julie chinmayi,intermediate,15 - 30 min
5,yoga foundations - non-attachment,0.958734,rest,julie chinmayi,beginner,6-15 min
6,yoga for concentration,0.990095,rest,julie chinmayi,intermediate,15 - 30 min
7,yoga for energy,0.990095,rest,julie chinmayi,intermediate,15 - 30 min
8,family friendly yoga with kids,1.025027,rest,julie chinmayi,beginner,6-15 min
9,yoga - the moon salutation,1.025027,rest,julie chinmayi,beginner,6-15 min


In [62]:
title = 'yoga foundations - concentration'
num_to_be_recommended = 10
metric = 'pearson'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,pearson,Pillar,Instructor,Difficulty,Duration
0,yoga foundations - concentration,1.000000,rest,julie chinmayi,beginner,6-15 min
1,yoga for cyclists by julie,0.799834,rest,julie chinmayi,beginner,6-15 min
2,yoga for grumpy knees,0.799834,rest,julie chinmayi,beginner,6-15 min
3,yoga for the lower back,0.799834,rest,julie chinmayi,beginner,6-15 min
4,yoga foundations - finding balance,0.548317,rest,julie chinmayi,intermediate,15 - 30 min
5,yoga foundations - non-attachment,0.535730,rest,julie chinmayi,beginner,6-15 min
6,yoga for concentration,0.504595,rest,julie chinmayi,intermediate,15 - 30 min
7,yoga for energy,0.504595,rest,julie chinmayi,intermediate,15 - 30 min
8,family friendly yoga with kids,0.468987,rest,julie chinmayi,beginner,6-15 min
9,yoga - the moon salutation,0.468987,rest,julie chinmayi,beginner,6-15 min


In [63]:
title = 'yoga foundations - concentration'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,yoga foundations - concentration,0.000000,rest,julie chinmayi,beginner,6-15 min
1,yoga for cyclists by julie,1.892909,rest,julie chinmayi,beginner,6-15 min
2,yoga for grumpy knees,1.892909,rest,julie chinmayi,beginner,6-15 min
3,yoga for the lower back,1.892909,rest,julie chinmayi,beginner,6-15 min
4,yoga foundations - non-attachment,3.655274,rest,julie chinmayi,beginner,6-15 min
5,what is yoga nidra,4.518836,rest,julie chinmayi,beginner,1-5 min
6,yoga for concentration,4.742385,rest,julie chinmayi,intermediate,15 - 30 min
7,yoga for energy,4.742385,rest,julie chinmayi,intermediate,15 - 30 min
8,family friendly yoga with kids,4.816403,rest,julie chinmayi,beginner,6-15 min
9,yoga - the moon salutation,4.816403,rest,julie chinmayi,beginner,6-15 min


# Example 3

In [64]:
title = 'guided meditation for achieving inner calm'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,deep relaxation guided meditation,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
1,guided meditation for achieving inner calm,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
2,guided meditation for achieving inner harmony,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
3,guided meditation for achieving inner peace,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
4,guided meditation for achieving inner serenity,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
5,guided meditation for achieving serenity,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
6,guided meditation for achieving tranquility,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
7,guided meditation for deep relaxation,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
8,guided meditation for deep tranquility,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
9,guided meditation for inner calm,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min


In [65]:
title = 'guided meditation for achieving inner calm'
num_to_be_recommended = 10
metric = 'euclidean'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,euclidean,Pillar,Instructor,Difficulty,Duration
0,deep relaxation guided meditation,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
1,guided meditation for achieving inner calm,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
2,guided meditation for achieving inner harmony,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
3,guided meditation for achieving inner peace,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
4,guided meditation for achieving inner serenity,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
5,guided meditation for achieving serenity,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
6,guided meditation for achieving tranquility,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
7,guided meditation for deep relaxation,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
8,guided meditation for deep tranquility,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
9,guided meditation for inner calm,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min


In [66]:
title = 'guided meditation for achieving inner calm'
num_to_be_recommended = 10
metric = 'pearson'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,pearson,Pillar,Instructor,Difficulty,Duration
0,deep relaxation guided meditation,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
1,guided meditation for achieving inner calm,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
2,guided meditation for achieving inner harmony,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
3,guided meditation for achieving inner peace,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
4,guided meditation for achieving inner serenity,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
5,guided meditation for achieving serenity,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
6,guided meditation for achieving tranquility,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
7,guided meditation for deep relaxation,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
8,guided meditation for deep tranquility,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
9,guided meditation for inner calm,1.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min


In [67]:
title = 'guided meditation for achieving inner calm'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,deep relaxation guided meditation,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
1,guided meditation for achieving inner calm,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
2,guided meditation for achieving inner harmony,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
3,guided meditation for achieving inner peace,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
4,guided meditation for achieving inner serenity,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
5,guided meditation for achieving serenity,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
6,guided meditation for achieving tranquility,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
7,guided meditation for deep relaxation,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
8,guided meditation for deep tranquility,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min
9,guided meditation for inner calm,0.0,"beauty, rest",julie chinmayi,beginner,15 - 30 min


# Example 4

In [43]:
title = 'cardio dance workout'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,cardio dance workout,1.000000,activity,walerija,intermediate,15 - 30 min
1,zumba fitness dance workout,0.587957,activity,walerija,intermediate,15 - 30 min
2,fun cardio dance party workout,0.340201,activity,walerija,intermediate,15 - 30 min
3,get moving with a fun cardio dance,0.332280,activity,walerija,intermediate,15 - 30 min
4,energetic cardio salsa workout,0.324662,activity,walerija,intermediate,15 - 30 min
5,energize your morning with a fun zumba workout,0.305253,activity,walerija,intermediate,15 - 30 min
6,energizing zumba dance workout,0.300987,activity,walerija,intermediate,15 - 30 min
7,hiit workout for beginners,0.279433,activity,walerija,beginner,15 - 30 min
8,yoga for enhancing flexibility and posture,0.278723,"rest, activity",walerija,intermediate,15 - 30 min
9,tai chi for enhancing balance and flexibility,0.251659,"rest, activity",walerija,intermediate,15 - 30 min


In [44]:
title = 'cardio dance workout'
num_to_be_recommended = 10
metric = 'euclidean'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,euclidean,Pillar,Instructor,Difficulty,Duration
0,cardio dance workout,0.000000,activity,walerija,intermediate,15 - 30 min
1,zumba fitness dance workout,0.907792,activity,walerija,intermediate,15 - 30 min
2,fun cardio dance party workout,1.148738,activity,walerija,intermediate,15 - 30 min
3,get moving with a fun cardio dance,1.155613,activity,walerija,intermediate,15 - 30 min
4,energetic cardio salsa workout,1.162186,activity,walerija,intermediate,15 - 30 min
5,energize your morning with a fun zumba workout,1.178768,activity,walerija,intermediate,15 - 30 min
6,energizing zumba dance workout,1.182382,activity,walerija,intermediate,15 - 30 min
7,hiit workout for beginners,1.200473,activity,walerija,beginner,15 - 30 min
8,yoga for enhancing flexibility and posture,1.201063,"rest, activity",walerija,intermediate,15 - 30 min
9,tai chi for enhancing balance and flexibility,1.223390,"rest, activity",walerija,intermediate,15 - 30 min


In [45]:
title = 'cardio dance workout'
num_to_be_recommended = 10
metric = 'pearson'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,pearson,Pillar,Instructor,Difficulty,Duration
0,cardio dance workout,1.000000,activity,walerija,intermediate,15 - 30 min
1,zumba fitness dance workout,0.583556,activity,walerija,intermediate,15 - 30 min
2,fun cardio dance party workout,0.331485,activity,walerija,intermediate,15 - 30 min
3,get moving with a fun cardio dance,0.323531,activity,walerija,intermediate,15 - 30 min
4,energetic cardio salsa workout,0.315784,activity,walerija,intermediate,15 - 30 min
5,energize your morning with a fun zumba workout,0.296089,activity,walerija,intermediate,15 - 30 min
6,energizing zumba dance workout,0.291774,activity,walerija,intermediate,15 - 30 min
7,hiit workout for beginners,0.271341,activity,walerija,beginner,15 - 30 min
8,yoga for enhancing flexibility and posture,0.268952,"rest, activity",walerija,intermediate,15 - 30 min
9,tai chi for enhancing balance and flexibility,0.241232,"rest, activity",walerija,intermediate,15 - 30 min


In [68]:
title = 'cardio dance workout'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,cardio dance workout,0.000000,activity,walerija,intermediate,15 - 30 min
1,zumba fitness dance workout,3.146018,activity,walerija,intermediate,15 - 30 min
2,hiit workout for beginners,6.775884,activity,walerija,beginner,15 - 30 min
3,fun cardio dance party workout,6.893287,activity,walerija,intermediate,15 - 30 min
4,get moving with a fun cardio dance,6.939438,activity,walerija,intermediate,15 - 30 min
5,cardio kickboxing workout,6.990688,activity,walerija,intermediate,30+ min
6,energetic cardio salsa workout,7.062481,activity,walerija,intermediate,15 - 30 min
7,energize your morning with a fun zumba workout,7.277765,activity,walerija,intermediate,15 - 30 min
8,yoga for improved flexibility,7.308128,"rest, activity",nitti,intermediate,15 - 30 min
9,energizing zumba dance workout,7.328980,activity,walerija,intermediate,15 - 30 min


# Example 5

In [13]:
title = 'the ultimate hiit challenge for fitness enthusiasts'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,the ultimate hiit challenge for fitness enthus...,1.000000,activity,nitti,expert,30+ min
1,the ultimate hiit workout for fitness enthusiasts,0.697857,activity,walerija,expert,30+ min
2,the ultimate hiit challenge for the fitness fa...,0.679411,activity,nitti,expert,30+ min
3,the ultimate hiit challenge for the fitness wa...,0.618033,activity,ola kwiatkowska,expert,30+ min
4,the ultimate hiit challenge for the fitness savvy,0.569519,activity,mark boateng,expert,30+ min
5,the ultimate hiit workout challenge,0.566042,activity,ola kwiatkowska,expert,30+ min
6,hiit workout: the ultimate challenge,0.557660,activity,mark boateng,expert,30+ min
7,the ultimate hiit workout for fitness lovers,0.471427,activity,walerija,expert,30+ min
8,hiit workout: the extreme challenge,0.470530,activity,walerija,expert,30+ min
9,hiit workout for advanced fitness,0.413699,activity,nitti,expert,30+ min


In [14]:
title = 'the ultimate hiit challenge for fitness enthusiasts'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,the ultimate hiit challenge for fitness enthus...,0.000000,activity,nitti,expert,30+ min
1,the ultimate hiit challenge for the fitness fa...,3.478466,activity,nitti,expert,30+ min
2,the ultimate hiit workout for fitness enthusiasts,3.607994,activity,walerija,expert,30+ min
3,the ultimate hiit challenge for the fitness wa...,4.435640,activity,ola kwiatkowska,expert,30+ min
4,the ultimate hiit challenge for the fitness savvy,5.024691,activity,mark boateng,expert,30+ min
5,the ultimate hiit workout challenge,5.161954,activity,ola kwiatkowska,expert,30+ min
6,hiit workout: the ultimate challenge,5.234775,activity,mark boateng,expert,30+ min
7,the ultimate hiit workout for fitness lovers,5.933474,activity,walerija,expert,30+ min
8,hiit workout: the extreme challenge,5.961537,activity,walerija,expert,30+ min
9,hiit workout for advanced fitness,6.387012,activity,nitti,expert,30+ min


# Example 6

In [15]:
title = 'affirmations for building confidence'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,affirmations for building confidence,1.000000,beauty,"jess vergeer, iain mcbride",beginner,1-5 min
1,building self-esteem with positive affirmations,0.533089,beauty,jess vergeer,beginner,1-5 min
2,empowerment through self-love affirmations,0.458874,beauty,iain mcbride,beginner,1-5 min
3,positive affirmations for self-love,0.345551,beauty,"jess vergeer, iain mcbride",beginner,1-5 min
4,affirmations for boosting self-confidence,0.333772,beauty,bao,beginner,1-5 min
5,affirmations for cultivating self-love,0.320753,beauty,jess vergeer,beginner,1-5 min
6,affirmations for embracing self-love,0.309260,beauty,bao,beginner,1-5 min
7,body positivity affirmations,0.250313,beauty,iain mcbride,beginner,1-5 min
8,positive affirmations for confidence,0.214091,beauty,iain mcbride,beginner,<1 min (shorts)
9,morning motivation affirmations,0.210888,beauty,iain mcbride,beginner,<1 min (shorts)


In [16]:
title = 'affirmations for building confidence'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,affirmations for building confidence,0.000000,beauty,"jess vergeer, iain mcbride",beginner,1-5 min
1,building self-esteem with positive affirmations,5.863479,beauty,jess vergeer,beginner,1-5 min
2,empowerment through self-love affirmations,7.209306,beauty,iain mcbride,beginner,1-5 min
3,positive affirmations for self-love,8.105716,beauty,"jess vergeer, iain mcbride",beginner,1-5 min
4,affirmations for boosting self-confidence,8.641734,beauty,bao,beginner,1-5 min
5,affirmations for cultivating self-love,8.665418,beauty,jess vergeer,beginner,1-5 min
6,affirmations for embracing self-love,8.894737,beauty,bao,beginner,1-5 min
7,body positivity affirmations,9.144923,beauty,iain mcbride,beginner,1-5 min
8,affirmations for self-esteem,9.407785,beauty,bao,beginner,1-5 min
9,positive affirmations for confidence,9.863695,beauty,iain mcbride,beginner,<1 min (shorts)


# Example 7

In [17]:
title = 'stress management techniques'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,stress management techniques,1.000000,"beauty, rest",iain mcbride,intermediate,15 - 30 min
1,mindfulness techniques for anxiety relief,0.387007,"beauty, rest",iain mcbride,intermediate,15 - 30 min
2,mindfulness techniques for anxiety,0.359566,"beauty, rest","jess vergeer, iain mcbride",intermediate,15 - 30 min
3,mindfulness techniques for emotional harmony,0.297309,"beauty, rest","jess vergeer, iain mcbride",intermediate,15 - 30 min
4,practicing mindfulness in daily life,0.293110,"beauty, rest","jess vergeer, iain mcbride",intermediate,15 - 30 min
5,mindfulness for stress relief,0.281963,"beauty, rest",walerija,intermediate,15 - 30 min
6,mindfulness practices for achieving inner balance,0.276904,"beauty, rest","jess vergeer, iain mcbride",intermediate,15 - 30 min
7,practicing mindfulness for stress relief,0.232947,"beauty, rest",bao,intermediate,15 - 30 min
8,mindfulness techniques for stress management,0.229277,"beauty, rest",jess vergeer,intermediate,15 - 30 min
9,mindfulness techniques for emotional resilience,0.211570,"beauty, rest",bao,intermediate,15 - 30 min


In [18]:
title = 'stress management techniques'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,stress management techniques,0.000000,"beauty, rest",iain mcbride,intermediate,15 - 30 min
1,mindfulness techniques for anxiety,7.013205,"beauty, rest","jess vergeer, iain mcbride",intermediate,15 - 30 min
2,mindfulness techniques for anxiety relief,7.408509,"beauty, rest",iain mcbride,intermediate,15 - 30 min
3,mindfulness for stress relief,7.559735,"beauty, rest",walerija,intermediate,15 - 30 min
4,mindfulness techniques for emotional harmony,8.806427,"beauty, rest","jess vergeer, iain mcbride",intermediate,15 - 30 min
5,practicing mindfulness in daily life,8.868316,"beauty, rest","jess vergeer, iain mcbride",intermediate,15 - 30 min
6,practicing mindfulness for stress relief,9.222994,"beauty, rest",bao,intermediate,15 - 30 min
7,mindfulness practices for achieving inner balance,9.286145,"beauty, rest","jess vergeer, iain mcbride",intermediate,15 - 30 min
8,mindfulness techniques for stress management,9.453266,"beauty, rest",jess vergeer,intermediate,15 - 30 min
9,mindfulness techniques for emotional resilience,9.456450,"beauty, rest",bao,intermediate,15 - 30 min


# Example 8

In [19]:
title = 'the science of plant-based proteins'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,the science of plant-based proteins,1.000000,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
1,the power of plant-based proteins,0.792643,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
2,the science behind plant-based proteins,0.596084,nutrition,jess vergeer,beginner,1-5 min
3,unraveling the health benefits of plant-based ...,0.592093,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
4,understanding the power of plant-based proteins,0.537653,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
5,demystifying the benefits of plant-based proteins,0.512722,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
6,understanding plant-based proteins,0.464644,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
7,exploring the benefits of plant-based proteins,0.446120,nutrition,iain mcbride,beginner,1-5 min
8,the benefits of plant-based proteins explained,0.432960,nutrition,jess vergeer,beginner,1-5 min
9,the power of plant-based proteins unveiled,0.431335,nutrition,jess vergeer,beginner,1-5 min


In [20]:
title = 'the science of plant-based proteins'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,the science of plant-based proteins,0.000000,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
1,the power of plant-based proteins,2.306276,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
2,unraveling the health benefits of plant-based ...,4.436877,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
3,the science behind plant-based proteins,4.919876,nutrition,jess vergeer,beginner,1-5 min
4,understanding the power of plant-based proteins,5.059585,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
5,demystifying the benefits of plant-based proteins,5.373709,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
6,understanding plant-based proteins,6.259996,nutrition,"jess vergeer, iain mcbride",beginner,1-5 min
7,exploring the benefits of plant-based proteins,6.595841,nutrition,iain mcbride,beginner,1-5 min
8,the power of plant-based proteins unveiled,6.759197,nutrition,jess vergeer,beginner,1-5 min
9,the benefits of plant-based proteins explained,6.761831,nutrition,jess vergeer,beginner,1-5 min


# Example 9

In [21]:
title = 'quick and easy plant-based recipes'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,quick and easy plant-based recipes,1.000000,nutrition,iain mcbride,beginner,6-15 min
1,quick and tasty plant-based meal recipes,0.421154,nutrition,iain mcbride,beginner,6-15 min
2,quick and healthy plant-based meal inspirations,0.408173,nutrition,iain mcbride,beginner,6-15 min
3,quick and healthy plant-based meal ideas,0.370587,nutrition,"jess vergeer, iain mcbride",beginner,6-15 min
4,quick and delicious plant-based meal ideas,0.359793,nutrition,"jess vergeer, iain mcbride",beginner,6-15 min
5,quick and nutritious plant-based recipes,0.358850,nutrition,iain mcbride,beginner,6-15 min
6,quick and healthy plant-based meal prep ideas,0.324789,nutrition,iain mcbride,beginner,6-15 min
7,fast and nutritious plant-based meal recipes,0.324707,nutrition,iain mcbride,beginner,6-15 min
8,delicious and nutritious plant-based recipes,0.323196,nutrition,iain mcbride,beginner,6-15 min
9,quick and nutritious plant-based meal ideas,0.286642,nutrition,jess vergeer,beginner,6-15 min


In [22]:
title = 'quick and easy plant-based recipes'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,quick and easy plant-based recipes,0.000000,nutrition,iain mcbride,beginner,6-15 min
1,quick and tasty plant-based meal recipes,6.632019,nutrition,iain mcbride,beginner,6-15 min
2,quick and healthy plant-based meal inspirations,6.655932,nutrition,iain mcbride,beginner,6-15 min
3,quick and nutritious plant-based recipes,6.861978,nutrition,iain mcbride,beginner,6-15 min
4,healthy green smoothie recipe,7.155852,nutrition,iain mcbride,beginner,<1 min (shorts)
5,hydrating smoothie recipe,7.234591,nutrition,iain mcbride,beginner,<1 min (shorts)
6,delicious and nutritious plant-based recipes,7.502351,nutrition,iain mcbride,beginner,6-15 min
7,quick and healthy plant-based meal prep ideas,7.517560,nutrition,iain mcbride,beginner,6-15 min
8,fast and nutritious plant-based meal recipes,7.518937,nutrition,iain mcbride,beginner,6-15 min
9,plant-based meal ideas,7.578278,nutrition,jess vergeer,beginner,6-15 min


# Example 10

In [23]:
title = 'drift into sleep with forest sounds'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,drift into sleep with forest sounds,1.000000,rest,iain mcbride,beginner,30+ min
1,sleep enhancing sounds of the rainforest,0.525955,rest,nitti,beginner,30+ min
2,restful sleep with ambient sounds,0.512059,rest,iain mcbride,beginner,30+ min
3,sleep soundly with soothing rain sounds,0.507562,rest,iain mcbride,beginner,30+ min
4,unwind with the sounds of a tranquil rainforest,0.488097,rest,"jess vergeer, iain mcbride",beginner,30+ min
5,soothing sleep with nature sounds,0.460041,rest,julie chinmayi,beginner,30+ min
6,relax with the sounds of a gentle rain,0.459639,rest,iain mcbride,beginner,30+ min
7,unwind with the sounds of a calming ocean,0.424617,rest,bao,beginner,30+ min
8,sleep soundly with ocean waves,0.421332,rest,sophon,beginner,30+ min
9,relax with the sounds of a serene waterfall,0.408108,rest,sophon,beginner,30+ min


In [24]:
title = 'drift into sleep with forest sounds'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,drift into sleep with forest sounds,0.000000,rest,iain mcbride,beginner,30+ min
1,sleep enhancing sounds of the rainforest,5.846083,rest,nitti,beginner,30+ min
2,sleep soundly with soothing rain sounds,5.892325,rest,iain mcbride,beginner,30+ min
3,restful sleep with ambient sounds,6.197216,rest,iain mcbride,beginner,30+ min
4,relax with the sounds of a gentle rain,6.657177,rest,iain mcbride,beginner,30+ min
5,unwind with the sounds of a tranquil rainforest,6.746571,rest,"jess vergeer, iain mcbride",beginner,30+ min
6,soothing sleep with nature sounds,6.928039,rest,julie chinmayi,beginner,30+ min
7,unwind with the sounds of a calming ocean,7.072904,rest,bao,beginner,30+ min
8,sleep soundly with ocean waves,7.142386,rest,sophon,beginner,30+ min
9,relax with the sounds of a serene waterfall,7.405386,rest,sophon,beginner,30+ min


# Example 11

In [25]:
title = 'low lunge - yoga library'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,boat pose,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
1,big toe posepadangusthasana,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
2,butterfly pose,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
3,chair pose,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
4,childs pose - yoga library,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
5,dancer pose,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
6,down dog - yoga library,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
7,eagle pose/garudasana,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
8,easy pose,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
9,forward fold,1.0,rest,ola kwiatkowska,beginner,<1 min (shorts)


In [26]:
title = 'low lunge - yoga library'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,boat pose,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
1,big toe posepadangusthasana,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
2,butterfly pose,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
3,chair pose,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
4,childs pose - yoga library,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
5,dancer pose,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
6,down dog - yoga library,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
7,eagle pose/garudasana,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
8,easy pose,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)
9,forward fold,0.0,rest,ola kwiatkowska,beginner,<1 min (shorts)


# Example 12

In [27]:
title = 'tai chi for enhancing balance and flexibility'
num_to_be_recommended = 10
metric = 'cosine'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,cosine,Pillar,Instructor,Difficulty,Duration
0,tai chi for enhancing balance and flexibility,1.000000,"rest, activity",walerija,intermediate,15 - 30 min
1,yoga for enhancing flexibility and posture,0.767743,"rest, activity",walerija,intermediate,15 - 30 min
2,yoga for enhancing flexibility and coordination,0.470428,"rest, activity",ola kwiatkowska,intermediate,15 - 30 min
3,yoga for enhancing flexibility and strength,0.395935,"rest, activity",nitti,intermediate,15 - 30 min
4,yoga for enhancing balance and strength,0.368035,"rest, activity","jess vergeer, iain mcbride",intermediate,15 - 30 min
5,yoga for improving posture and flexibility,0.354004,"rest, activity",bao,intermediate,15 - 30 min
6,yoga for improving balance and flexibility,0.314315,"rest, activity","jess vergeer, iain mcbride",intermediate,15 - 30 min
7,restorative yoga session,0.290689,"rest, activity",walerija,beginner,30+ min
8,yoga for enhanced balance and strength,0.278908,"rest, activity",ola kwiatkowska,intermediate,15 - 30 min
9,yoga for enhancing agility and strength,0.259494,"rest, activity",ola kwiatkowska,intermediate,15 - 30 min


In [28]:
title = 'tai chi for enhancing balance and flexibility'
num_to_be_recommended = 10
metric = 'manhattan'
recommend_similar(title, num_to_be_recommended, metric)

,Recommended_titles,manhattan,Pillar,Instructor,Difficulty,Duration
0,tai chi for enhancing balance and flexibility,0.000000,"rest, activity",walerija,intermediate,15 - 30 min
1,yoga for enhancing flexibility and posture,3.124009,"rest, activity",walerija,intermediate,15 - 30 min
2,yoga for enhancing flexibility and coordination,6.514259,"rest, activity",ola kwiatkowska,intermediate,15 - 30 min
3,yoga for enhancing flexibility and strength,7.451951,"rest, activity",nitti,intermediate,15 - 30 min
4,yoga for improving posture and flexibility,7.761619,"rest, activity",bao,intermediate,15 - 30 min
5,yoga for enhancing balance and strength,8.021088,"rest, activity","jess vergeer, iain mcbride",intermediate,15 - 30 min
6,restorative yoga session,8.216127,"rest, activity",walerija,beginner,30+ min
7,cardio dance workout,8.302935,activity,walerija,intermediate,15 - 30 min
8,zumba fitness dance workout,8.309436,activity,walerija,intermediate,15 - 30 min
9,yoga for improving balance and flexibility,8.605129,"rest, activity","jess vergeer, iain mcbride",intermediate,15 - 30 min


Cosine, Euclidean & Pearson output similar titles & their order but Manhattan outputs somewhat different titles & also the order is different.